In [22]:
import os
os.environ["ISISROOT"]="/opt/conda/envs/isis/"
os.environ["ISISDATA"]="/isis/data"
import fnmatch
import kalasiris as isis
from kalasiris import hi2isis, hical, histitch, spiceinit, spicefit, noproj, hijitreg, handmos, cubenorm, cam2map
import psutil
import re
import shutil
import subprocess
from tqdm import tqdm
JOBS = psutil.cpu_count()

**Definition of a function to list all files with a specific keyword or extension in a defined folder**

In [21]:
def get_paths(PATH, ixt):
    ext=f'*{ixt}*'
    chkCase = re.compile(fnmatch.translate(ext), re.IGNORECASE)
    files = [PATH+'/'+f for f in os.listdir(PATH) if chkCase.match(f)]
    return(sorted(files))

**Definition of the main function used to parallelize all the processing functions**

In [ ]:
def parallel_1(files, JOBS, proc_fun, **kwargs):
    from joblib import Parallel, delayed, parallel_backend
    with parallel_backend("loky", inner_max_num_threads=2):

        Parallel (n_jobs=JOBS)(delayed(proc_fun)(files[i],**kwargs)
                                for i in range(len(files)))

## Define user variables
**Here we define:**
- the source folder where the HiRISE RED CCD files are contained
- the destination folder (if it necessary to change)
- the maptemplated that will be used for the map projection
- the del_temp flag used to delete intermediate files (saving a lot of space)

In [ ]:
source = '/home/jovyan/Data/ESP_068426_1985'
destination = source
maptemplate = '/home/jovyan/Data/PyISIS-Parallel/PyISIS-Parallel/maptemplates/CenterEquirectangularMars.map'
ixt = 'IMG'
oxt='tiff'
delete = True

# Get file list
**Here we create a list of all CCD files**

In [ ]:
ccd_list = get_paths(source, ixt) 

**tmp folder creation**

In [ ]:
tmp_dir = f"{destination}/tmp/"
os.makedirs(f"{tmp_dir}", exist_ok = True)

# Parallel conversion of CCD to cub (hi2isis)
**We define the function that convert the EDR IMG files into ISIS CUB files using ISIS *hi2isis* command**

In [ ]:
def h2i(src):
    src_basename = os.path.basename(src).split('.'+ixt)[0]
    dst_basename = f"{tmp_dir}{src_basename}"
    L0 = dst_basename+'_lev0.cub'
    if not os.path.isfile(L0):
        try:
            h2 = hi2isis(src, to=L0)            
            return h2
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            raise err
            print(err)
            return err
            

**Parallel execution of the h2i function**

In [ ]:
with tqdm(total=len(ccd_list),
             desc = 'Generating L0 cubs',
             unit='File') as pbar:
        filerange = len(ccd_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(ccd_list, JOBS):
            chunks.append(c)
        for i in range(len(chunks)):
            files = chunks[i]
            parallel_1(files, JOBS, h2i)
            pbar.update(len(files))

## Parallel Calibration
**We define the function that calibrate each CCD cube using ISIS *hical* command**

In [ ]:
def h2cal(src, del_tmp):
    src_basename = os.path.basename(src).split('_lev0.cub')[0]
    dst_basename = f"{tmp_dir}{src_basename}"
    L1 = dst_basename+'_lev1.cub'       
    if not os.path.isfile(L1):
        try:
            h2c = hical(src, to=L1)
            if del_tmp:
                os.remove(src)        
            return h2c
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            raise err
            return err

**Parallel execution of the hical function**

In [ ]:
L0_list = get_paths(tmp_dir, 'lev0') 
with tqdm(total=len(L0_list),
             desc = 'Generating L1 cubs',
             unit='File') as pbar:
        filerange = len(L0_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(L0_list, JOBS):
            chunks.append(c)
        for i in range(len(chunks)):
            files = chunks[i]
            parallel_1(files, JOBS, h2cal, del_tmp=delete)
            pbar.update(len(files))

## Parallel stitch
**We define the function that stitch each CCD pair using ISIS histitch command**

In [ ]:
def h2s(channel_pair, del_tmp):
    chan_prefix = os.path.commonprefix(channel_pair)
    chan_suffix = 'histich.cub'
    dst = f'{chan_prefix}{chan_suffix}'
    if not os.path.isfile(dst):
        try:
            h2 = histitch(balance=True, from1=channel_pair[0], from2=channel_pair[1], to=dst)
            if del_tmp:
                os.remove(channel_pair[0])
                os.remove(channel_pair[1])
            return h2
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            raise err
            return err

**We generate a list of CCD Pairs**

In [ ]:
L1_list = get_paths(tmp_dir, 'lev1') 
# function from source hiedr2mosaic.py https://github.com/NeoGeographyToolkit/StereoPipeline/blob/master/src/asp/Tools/hiedr2mosaic.py
import re
prefix = os.path.commonprefix( L1_list )
channel_files = [[None]*2 for i in range(len(L1_list)//2)]
pattern = re.compile(r"(\d)_(\d)")
for cub in L1_list:
    match = re.match( pattern, cub[len(prefix):] )
    if match:
        ccd     = match.group(1)
        channel = match.group(2)
        # print ('ccd: ' + ccd + ' channel: '+ channel)
        channel_files[int(ccd)][int(channel)] = cub
    else:
        raise Exception( 'Could not find a CCD and channel identifier in ' + cub )

**Parallel execution of the histich function**

In [ ]:
with tqdm(total=len(channel_files),
             desc = 'Stitching channels',
             unit='File') as pbar:
        filerange = len(channel_files)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(channel_files, JOBS):
            chunks.append(c)
        for i in range(len(chunks)):
            files = chunks[i]
            parallel_1(files, JOBS, h2s, del_tmp=delete)
            pbar.update(len(files))

# Parallel Spiceinit
**We define the function that initialize the retreive the acqusition parameters for each CCD**

In [ ]:
def spice(src, web):
    import subprocess
    err=None
    init = None
    ii = 0
    while init == None:
        try:
            sp = spiceinit(src, web=web)
            init = 'Done'
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            ii+=1
            if ii==100:
                break
                raise err
            raise err
 
    return([sp, err])

**Parallel execution of the spiceinit function**

In [ ]:
stitched_list = get_paths(tmp_dir, 'histich') 
with tqdm(total=len(stitched_list),
             desc = 'Spiceinit',
             unit='File') as pbar:
        filerange = len(stitched_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(stitched_list, JOBS//2):
            chunks.append(c)
        for i in range(len(chunks)):
            files = chunks[i]
            parallel_1(files, JOBS//2, spice, web=True)
            pbar.update(len(files))

# Parallel noproj
**We define the function that removes the camera distortions each stitchedCCD using ISIS *noproj* command**

In [ ]:
def noprj(src, mtch, del_tmp):
    base_dir = os.path.dirname(src)
    basename = os.path.basename(src).split('.cub')[0]
    #tmp_dir = f"{base_dir}/tmp_{basename}/"
    #os.makedirs(tmp_dir, exist_ok=True)    
    dst_noproj = f"{src.split('histich')[0]}noproj.cub"
    if not os.path.isfile(dst_noproj):
        try:    
            npj = noproj(src, match=mtch, source='frommatch',to=dst_noproj)
            if del_tmp:
                os.remove(src)
            #noproj from=../ESP_068426_1985_RED0.histitch.cub match=../ESP_068426_1985_RED5.histitch.cub source= frommatch to=../ESP_068426_1985_RED0.noproj.cub
            return npj
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            raise err
            return err

**Parallel execution of the noproj function**

In [ ]:
match_id=5
with tqdm(total=len(stitched_list),
             desc = 'Generating noproj',
             unit='File') as pbar:
        filerange = len(stitched_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(stitched_list, JOBS):
            chunks.append(c)
        for i in range(len(chunks)):
            files = chunks[i]
            parallel_1(files, JOBS, noprj, mtch=stitched_list[match_id], del_tmp=delete)
            pbar.update(len(files))

# Parallel hijitreg
**We define the function that characterize HiRISE jitter with co-registration using ISIS *hijitreg* command**

In [ ]:
def hiji(srcs, del_tmp):
    src, mtch = srcs
    basename = f"{src.split('noproj')[0]}"
    flat_file = f"{basename}flat.txt"
    if not os.path.isfile(flat_file):
        try:    
            hij = hijitreg(src, match=mtch, flat=flat_file)  
            if del_tmp:
                os.remove(src)
            return hij
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            raise err
            return err

**Parallel execution of the hijitreg function**

In [ ]:
noproj_list = sorted(get_paths(tmp_dir, 'noproj'))
with tqdm(total=len(noproj_list),
             desc = 'Performing hijitreg',
             unit='File') as pbar:
        filerange = len(noproj_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(noproj_list, JOBS):
            chunks.append(c)
        for i in range(len(chunks)):
            files = chunks[i]
            matches = [files[i+1] for i in range(len(files)-1)]
            zipped = list(zip(files,matches))
            parallel_1(zipped, JOBS, hiji, del_tmp=False)
            pbar.update(len(files))

# Generating Mosaic
**Finally, we assemble the first mosaic using the noproj cubs and associated flat_files which contains co-registration values.**

**Definition of a function to read flat_file (from original hieadr2mosaic.py)**

In [ ]:
# function from source hiedr2mosaic.py https://github.com/NeoGeographyToolkit/StereoPipeline/blob/master/src/asp/Tools/hiedr2mosaic.py
def read_flatfile( flat ):
    f = open(flat,'r')
    averages = [0.0,0.0]
    try:
        for line in f:
            if line.rfind("Average Sample Offset:") > 0:
                index       = line.rfind("Offset:")
                index_e     = line.rfind("StdDev:")
                crop        = line[index+7:index_e]
                averages[0] = float(crop)
            elif line.rfind("Average Line Offset:") > 0:
                index       = line.rfind("Offset:")
                index_e     = line.rfind("StdDev:")
                crop        = line[index+7:index_e]
                averages[1] = float(crop)
    except ValueError:
        print("Could not extract valid offsets from the flat file (" +
              flat + "). "
              "This could be because no matches were found. "
              "You may need to run hijitreg manually with a "
              "custom REGDEF parameter.  In order for this program "
              "to complete, we are returning zeros as the offset "
              "but this may result in misaligned CCDs.")
    return averages

**Definition of a function to assemble the mosaic using noproj cubs and flat_files**

In [ ]:
def mosaic(noproj_list, prefix, match_id,averages ):
    #derived from function from source hiedr2mosaic.py https://github.com/NeoGeographyToolkit/StereoPipeline/blob/master/src/asp/Tools/hiedr2mosaic.py
    mosaic_name = f"{prefix}_mos_hijitreged.cub"
    if not os.path.isfile(mosaic_name):
        try:        
            shutil.copy(noproj_list[match_id], mosaic_name )
            sample_sum = 1
            line_sum   = 1
            for i in range( match_id-1, -1, -1):

                sample_sum  += averages[i][0]
                line_sum    += averages[i][1]
                handmos(noproj_list[i],
                        mosaic=mosaic_name,
                        outsample=str(int(round(sample_sum))),
                        outline=str( int(round(line_sum))),
                        priority='beneath')

            sample_sum = 1
            line_sum = 1
            for i in range( match_id+1, len(noproj_list), 1):

                sample_sum  -= averages[i-1][0]
                line_sum    -= averages[i-1][1]
                handmos(noproj_list[i],
                        mosaic=mosaic_name,
                        outsample=str(int(round(sample_sum))),
                        outline=str( int(round(line_sum))),
                        priority='beneath')
            return mosaic_name
        except subprocess.CalledProcessError as err:
            print(err.stdout)
            print(err.stderr)
            raise err
            return err
    else:
        print('Already Processed')
        return mosaic_name

**Creation of flat_files list, to be used with mosaic function**

In [ ]:
flat_list = sorted(get_paths(source, 'flat'))
averages = dict()
for i in range(len(flat_list)):
    averages[i] = read_flatfile(flat_list[i] )

**Generation of the first mosaic**

In [ ]:

mosaicked = mosaic( noproj_list, prefix=os.path.commonprefix(ccd_list),match_id=match_id, averages=averages )

**Mosaic Normalization**

In [ ]:
dst_norm = f"{os.path.commonprefix(ccd_list)}_norm.cub"
if not os.path.isfile(dst_norm):
    cubenorm(mosaicked, to=dst_norm)
else:
    print('Already Processed')

# Cleanup of temporary files

In [ ]:
shutil.rmtree(tmp_dir)

# Map projection

In [ ]:
dst_map = f"{dst_norm.split('.cub')[0]}_map.cub"
if not os.path.isfile(dst_map):
    try:    
        cam2map(dst_norm, to=dst_map, map=maptemplate)
        os.remove(dst_norm)
    except subprocess.CalledProcessError as err:
        print(err.stdout)
        print(err.stderr)
        raise err

In [ ]:
from utils.KalaUtils import L2toStd

In [ ]:
dst_std = f"{dst_norm.split('cub')[0]}tiff"
if not os.path.isfile(dst_std):
    try:
        L2toStd(dst_map, dst_std, byte='N')
    except Exception as err:
        print(err)